In [23]:
#Inspiration :D
#https://gitlab.com/ryanlogsdon/pubsub-for-python/-/blob/main/subscriber.py
#https://www.youtube.com/watch?v=xOtrCmPjal8&ab_channel=D-I-Ry
import os
from google.cloud import pubsub_v1
import sys
import time
import pandas as pd
import http.client
import datetime
from io import StringIO
import functools as ft

In [2]:
! python --version

Python 3.7.4


## Create connection to PubSub

In [60]:
credentials_path = 'weather-based-forecasting-v2-c4bde37656a7.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

In [61]:
publisher_obs = pubsub_v1.PublisherClient(
    publisher_options = pubsub_v1.types.PublisherOptions(
        enable_message_ordering=True,
    )
)

publisher_forecast = pubsub_v1.PublisherClient(
    publisher_options = pubsub_v1.types.PublisherOptions(
        enable_message_ordering=True,
    )
)

topic_path_obs = 'projects/weather-based-forecasting-v2/topics/observations_topic'
topic_path_forecast = 'projects/weather-based-forecasting-v2/topics/forecasts_topic'


## Download the data

In [62]:
time.sleep(5)

locations = ["Berlin", "Hamburg","Bremen"]


conn = http.client.HTTPSConnection("visual-crossing-weather.p.rapidapi.com")

headers = {
    "X-RapidAPI-Key": "c7b48b7c4emsh8a3b688e71b9896p1be4d0jsnda2e5b27b486",
    "X-RapidAPI-Host": "visual-crossing-weather.p.rapidapi.com"
}

df_list = []
for location in locations:

    conn.request("GET", "/forecast?location={}&aggregateHours=1&shortColumnNames=0&unitGroup=metric&contentType=csv".format(location),
                 headers=headers)

    res = conn.getresponse()
    data = res.read()
    s = str(data, 'utf-8')
    data_string = StringIO(s)
    df = pd.read_csv(data_string)
    # df.to_csv("./weather_prediction_{}.csv".format(location))
    df = df.add_suffix("_{}".format(location))
    df["Date time"] = df["Date time_{}".format(location)]
    df = df.drop(columns=["Date time_{}".format(location)])
    df_list.append(df)

In [63]:
df_merged = ft.reduce(lambda left, right: pd.merge(left, right, on='Date time'), df_list)
df_merged

,Address_Berlin,Latitude_Berlin,Longitude_Berlin,Resolved Address_Berlin,Name_Berlin,Wind Direction_Berlin,Temperature_Berlin,Wind Speed_Berlin,Cloud Cover_Berlin,Heat Index_Berlin,...,Heat Index_Bremen,Chance Precipitation (%)_Bremen,Precipitation_Bremen,Sea Level Pressure_Bremen,Snow Depth_Bremen,Snow_Bremen,Relative Humidity_Bremen,Wind Gust_Bremen,Wind Chill_Bremen,Conditions_Bremen
0,Berlin,52.516,13.3769,"Berlin, Deutschland",Berlin,131.7,-4.2,10.1,85.9,NaN,...,NaN,0.0,0.0,1026.0,0.0,0.0,73.9,28.8,-8.3,Overcast
1,Berlin,52.516,13.3769,"Berlin, Deutschland",Berlin,135.0,-4.4,10.1,96.9,NaN,...,NaN,0.0,0.0,1026.0,0.0,0.0,71.6,29.9,-8.6,Overcast
2,Berlin,52.516,13.3769,"Berlin, Deutschland",Berlin,133.8,-4.6,10.4,100.0,NaN,...,NaN,0.0,0.0,1025.0,0.0,0.0,71.6,28.1,-8.6,Overcast
3,Berlin,52.516,13.3769,"Berlin, Deutschland",Berlin,138.0,-4.7,10.8,100.0,NaN,...,NaN,0.0,0.0,1025.0,0.0,0.0,72.2,28.8,-8.7,Overcast
4,Berlin,52.516,13.3769,"Berlin, Deutschland",Berlin,143.1,-4.9,10.4,100.0,NaN,...,NaN,0.0,0.0,1024.0,0.0,0.0,72.2,28.4,-8.5,Overcast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,Berlin,52.516,13.3769,"Berlin, Deutschland",Berlin,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
382,Berlin,52.516,13.3769,"Berlin, Deutschland",Berlin,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
383,Berlin,52.516,13.3769,"Berlin, Deutschland",Berlin,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
384,Berlin,52.516,13.3769,"Berlin, Deutschland",Berlin,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
df2 = df_merged.copy()
df2.index = pd.to_datetime(df2["Date time"])
df2 = df2.asfreq(freq="1min")
df2 = df2.interpolate(method ='linear', limit_direction ='forward')

for location in locations:
    df2["Conditions_{}".format(location)] = df2["Conditions_{}".format(location)].fillna(method="backfill")
    df2["Address_{}".format(location)] = df2["Address_{}".format(location)][0]
    df2["Resolved Address_{}".format(location)] = df2["Name_{}".format(location)][0]
    df2["Name_{}".format(location)] = df2["Name_{}".format(location)][0]
df2["Date time"] = df2.index

# we're taking 10 days of a forecast
# now = datetime.datetime.now()
# minutes = now.minute
# df3 = df2.iloc[minutes-1:60*24*10, :]
df3 = df2.iloc[:60*24*10, :]
#df3.to_csv(sys.stdout, index=False)

In [65]:
now = datetime.datetime.now()
minutes = now.minute
df3_hour = df3.iloc[minutes-1:minutes+59, :]
df3_hour.head(5)

,Address_Berlin,Latitude_Berlin,Longitude_Berlin,Resolved Address_Berlin,Name_Berlin,Wind Direction_Berlin,Temperature_Berlin,Wind Speed_Berlin,Cloud Cover_Berlin,Heat Index_Berlin,...,Heat Index_Bremen,Chance Precipitation (%)_Bremen,Precipitation_Bremen,Sea Level Pressure_Bremen,Snow Depth_Bremen,Snow_Bremen,Relative Humidity_Bremen,Wind Gust_Bremen,Wind Chill_Bremen,Conditions_Bremen
Date time,,,,,,,,,,,,,,,,,,,,,
2022-12-18 17:58:00,Berlin,52.516,13.3769,Berlin,Berlin,134.890,-4.393333,10.100,96.533333,NaN,...,NaN,0.0,0.0,1026.000000,0.0,0.0,71.676667,29.863333,-8.590,Overcast
2022-12-18 17:59:00,Berlin,52.516,13.3769,Berlin,Berlin,134.945,-4.396667,10.100,96.716667,NaN,...,NaN,0.0,0.0,1026.000000,0.0,0.0,71.638333,29.881667,-8.595,Overcast
2022-12-18 18:00:00,Berlin,52.516,13.3769,Berlin,Berlin,135.000,-4.400000,10.100,96.900000,NaN,...,NaN,0.0,0.0,1026.000000,0.0,0.0,71.600000,29.900000,-8.600,Overcast
2022-12-18 18:01:00,Berlin,52.516,13.3769,Berlin,Berlin,134.980,-4.403333,10.105,96.951667,NaN,...,NaN,0.0,0.0,1025.983333,0.0,0.0,71.600000,29.870000,-8.600,Overcast
2022-12-18 18:02:00,Berlin,52.516,13.3769,Berlin,Berlin,134.960,-4.406667,10.110,97.003333,NaN,...,NaN,0.0,0.0,1025.966667,0.0,0.0,71.600000,29.840000,-8.600,Overcast


## Send observations in intervals to PubSub - check Weather_PubSub_check for results

In [67]:
for i in range(12):
    obs = str(df3_hour.iloc[i,:].astype(str).to_dict())
    obs = obs.replace("'", '"')
    index = df3_hour.iloc[i,:].name
    index = str(int(index.value / 10**9)).encode("utf-8")
    obs = obs.encode("utf-8")
    future = publisher_obs.publish(topic_path_obs, obs, ordering_key=index)
    print(f'published message id {future.result()}')
    time.sleep(60)

published message id 6606555072800292
published message id 6606556375058196
published message id 6606561936228687
published message id 6606556375130949
published message id 6606556513681138
published message id 6606556523481275
published message id 6606556397188577
published message id 6606556397331773
published message id 6606556397272029
published message id 6606555798198919
published message id 6606556410589864
published message id 6606556543697942


## Send Forecast in intervals to PubSub

In [16]:
df3_forecast = df3.iloc[minutes-1:, :]
df3_forecast.index = df3_forecast.index.astype(str)
for i in range(60):
    
    df3_forecast_window = str(df3_forecast.iloc[i:i+7*24*60,:].astype(str).to_dict("index"))
#     for key in df3_forecast_window.keys():
#         df3_forecast_window[key] = str(df3_forecast_window[key])
    df3_forecast_window = df3_forecast_window.encode('utf-8')
        
    info = "Weather forecast for Hamburg, {}".format(df3_forecast.index[i])
    info = info.encode('utf-8')
    
    future = publisher_forecast.publish(topic_path_forecast, df3_forecast_window)
    print(f'published message id {future.result()}')
    time.sleep(60)

published message id 6606510957785406
published message id 6606513623280419


KeyboardInterrupt: 